In [3]:
import scipy
import numpy as np

In [4]:
from scipy.sparse import lil_matrix

In [5]:
# Generating unrotated planar surface code of distance d
def generate_unrotated_surface_code(d: int):
    assert d > 2, "Distance must be greater than 2"
    
    # For unrotated surface code with distance 'd' the number of physical qubits is 2d(d- 1)
    num_qubits = 2*d*(d-1)

    # Horizontal edges: (row i in [0..d-1], col j in [0..d-2]) between (i,j) -- (i,j+1)
    horiz_id = {}
    vid = 0
    for i in range(d):
        for j in range(d-1):
            horiz_id[(i, j)] = vid
            vid += 1
    # Vertical edges: (row i in [0..d-2], col j in [0..d-1]) between (i,j) -- (i+1,j)
    vert_id = {}
    for i in range(d-1):
        for j in range(d):
            vert_id[(i, j)] = vid
            vid += 1
    if num_qubits !=vid:
        raise ValueError('Something is wrong')
        

    # HZ matrix
    # For HZ matrix the number of rows is equal to the number of faces
    num_faces = (d - 1)*(d - 1)
    HZ = lil_matrix((num_faces, num_qubits), dtype=int)

    row = 0 


    for i in range(d - 1):
        for j in range(d - 1):
            HZ[row, horiz_id[(i, j)]] = 1 #top
            HZ[row, vert_id[(i, j+1)]] = 1 #right
            HZ[row, horiz_id[(i+1, j)]] = 1 #bottom
            HZ[row, vert_id[(i, j)]] = 1 #left

            row += 1

        HZ = HZ.tocsr()

    # For HX matrix the number of rows is equal to the number of vertices
    num_vertices = d*d
    HX = lil_matrix((num_vertices, num_qubits), dtype=int)

    row = 0
    for i in range(d):
        for j in range(d):
            # incident horizontal edges
            if j > 0:
                HX[row, horiz_id[(i, j-1)]] = 1   # left
            if j < d-1:
                HX[row, horiz_id[(i, j)]] = 1     # right
            # incident vertical edges
            if i > 0:
                HX[row, vert_id[(i-1, j)]] = 1    # up
            if i < d-1:
                HX[row, vert_id[(i, j)]] = 1      # down
            row += 1

    HX = HX.tocsr()

    # Checking CSS condition
    if np.any((HX @ HZ.T).toarray() % 2 != 0):
        raise ValueError("CSS condition violated")

    return HZ, HX, num_faces, num_qubits, num_vertices

In [6]:
HZ, HX, num_faces, num_qubits, num_vertices = generate_unrotated_surface_code(10)

C:\Users\IISER13\AppData\Roaming\Python\Python313\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


In [7]:
print("HZ matrix:")
print(HZ.toarray())
print("HX matrix:")
print(HX.toarray())
print("Number of faces:", num_faces)
print("Number of qubits:", num_qubits)
print("Number of vertices:", num_vertices)

HZ matrix:
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 0 1 1]]
HX matrix:
[[1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
Number of faces: 81
Number of qubits: 180
Number of vertices: 100
